In [1]:
# import dependencies
import pandas as pd
import glob

In [2]:
# Fiind all csv files in Resources folder
path = 'Resources'
allFiles = glob.glob(path + '/*.csv')

In [3]:
allFiles

['Resources/JC-201801-citibike-tripdata.csv',
 'Resources/JC-201809-citibike-tripdata.csv',
 'Resources/JC-201803-citibike-tripdata.csv',
 'Resources/JC-201805-citibike-tripdata.csv',
 'Resources/JC-201811-citibike-tripdata.csv',
 'Resources/JC-201807-citibike-tripdata.csv',
 'Resources/JC-201808-citibike-tripdata.csv',
 'Resources/JC-201802-citibike-tripdata.csv',
 'Resources/JC-201804-citibike-tripdata.csv',
 'Resources/JC-201812-citibike-tripdata.csv',
 'Resources/JC-201806-citibike-tripdata.csv',
 'Resources/JC-201810-citibike-tripdata.csv']

In [4]:
# empty list to hold dfs from csv
df_list = []

# Loop through all csv files and read them as pandas df, then append tio list of df_list
for filePath in allFiles:
    df = pd.read_csv(filePath, index_col=None, header=0)
    df_list.append(df)

In [5]:
# concactenate all dfs

df = pd.concat(df_list, axis=0, ignore_index=True)

In [6]:
df.tail()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
353887,1118,2018-10-31 11:30:53.7070,2018-10-31 11:49:32.4850,3694,Jackson Square,40.71113,-74.0789,3185,City Hall,40.717733,-74.043845,26219,Subscriber,1991,1
353888,1242,2018-10-31 12:20:41.6990,2018-10-31 12:41:24.4270,3694,Jackson Square,40.71113,-74.0789,3185,City Hall,40.717733,-74.043845,26288,Customer,1986,1
353889,1188,2018-10-31 12:21:36.2790,2018-10-31 12:41:24.3540,3694,Jackson Square,40.71113,-74.0789,3185,City Hall,40.717733,-74.043845,29591,Customer,1977,1
353890,983,2018-10-31 14:27:59.9590,2018-10-31 14:44:23.2020,3694,Jackson Square,40.71113,-74.0789,3277,Communipaw & Berry Lane,40.714358,-74.066611,32831,Subscriber,1988,1
353891,544,2018-10-31 15:37:41.3760,2018-10-31 15:46:45.5090,3694,Jackson Square,40.71113,-74.0789,3268,Lafayette Park,40.713464,-74.062859,29490,Subscriber,1991,2


In [7]:
# Goal is to create a trip id column for each unique trip, then seperate the start and end stations into seperate rows. The indes is basically a trip id column
df = df.reset_index()

# rename index column to tripID
df = df.rename(columns={'index':'tripID'})

# Create path id column
df['pathID'] = df['start station name'] + '-' + df['end station name']

df.head()

,tripID,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,pathID
0,0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,Exchange Place-Newport Pkwy
1,1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,Exchange Place-Newport Pkwy
2,2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1,Exchange Place-Newport Pkwy
3,3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1,Exchange Place-Morris Canal
4,4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1,Exchange Place-Harborside


In [8]:
df.columns

Index(['tripID', 'tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'pathID'],
      dtype='object')

In [9]:
# seperate dfs into start and end

start_df = df[['tripID', 'tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'pathID']].copy()

# rename columns to remove 'start'
start_df.columns = start_df.columns.str.replace('start ','')

# add origin-destination column
start_df['origin-destination'] = 'origin'

start_df.head()

,tripID,tripduration,starttime,stoptime,station id,station name,station latitude,station longitude,bikeid,usertype,birth year,gender,pathID,origin-destination
0,0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3183,Exchange Place,40.716247,-74.033459,31929,Subscriber,1992,1,Exchange Place-Newport Pkwy,origin
1,1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3183,Exchange Place,40.716247,-74.033459,31845,Subscriber,1969,2,Exchange Place-Newport Pkwy,origin
2,2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3183,Exchange Place,40.716247,-74.033459,31708,Subscriber,1946,1,Exchange Place-Newport Pkwy,origin
3,3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,3183,Exchange Place,40.716247,-74.033459,31697,Subscriber,1994,1,Exchange Place-Morris Canal,origin
4,4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,3183,Exchange Place,40.716247,-74.033459,31861,Subscriber,1991,1,Exchange Place-Harborside,origin


In [10]:
end_df = df[['tripID', 'tripduration', 'starttime', 'stoptime', 'end station id',
        'end station name', 'end station latitude', 'end station longitude', 
        'bikeid', 'usertype', 'birth year', 'gender', 'pathID']].copy()

# rename columns to remove 'end '
end_df.columns = end_df.columns.str.replace('end ','')

# add origin-destination column
end_df['origin-destination'] = 'destination'

end_df.head()

,tripID,tripduration,starttime,stoptime,station id,station name,station latitude,station longitude,bikeid,usertype,birth year,gender,pathID,origin-destination
0,0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,Exchange Place-Newport Pkwy,destination
1,1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,Exchange Place-Newport Pkwy,destination
2,2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1,Exchange Place-Newport Pkwy,destination
3,3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1,Exchange Place-Morris Canal,destination
4,4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1,Exchange Place-Harborside,destination


In [11]:
# concactenate dataframes
processed_df = pd.concat([start_df, end_df], axis=0)
processed_df = processed_df.sort_values(['tripID', 'origin-destination'], ascending=[True,False])
processed_df.head()

,tripID,tripduration,starttime,stoptime,station id,station name,station latitude,station longitude,bikeid,usertype,birth year,gender,pathID,origin-destination
0,0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3183,Exchange Place,40.716247,-74.033459,31929,Subscriber,1992,1,Exchange Place-Newport Pkwy,origin
0,0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,Exchange Place-Newport Pkwy,destination
1,1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3183,Exchange Place,40.716247,-74.033459,31845,Subscriber,1969,2,Exchange Place-Newport Pkwy,origin
1,1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,Exchange Place-Newport Pkwy,destination
2,2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3183,Exchange Place,40.716247,-74.033459,31708,Subscriber,1946,1,Exchange Place-Newport Pkwy,origin


In [12]:
# save to df
processed_df.to_csv('Resources/Processed_data/processed_trip_data.csv')